In [1]:
import pandas as pd
import os
%pwd

u'/Users/Driz/capstone'

In [2]:
df = pd.read_csv('gender_included_df.csv', encoding='utf-8')

/Users/Driz/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (4,5,6,13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.head()

,id,episode_id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,spoken_words,normalized_text,word_count,gender
0,9549,32,209,"Miss Hoover: No, actually, it was a little of ...",848000,True,464,3.0,Miss Hoover,Springfield Elementary School,"No, actually, it was a little of both. Sometim...",no actually it was a little of both sometimes ...,31.0,f
1,9550,32,210,Lisa Simpson: (NEAR TEARS) Where's Mr. Bergstrom?,856000,True,9,3.0,Lisa Simpson,Springfield Elementary School,Where's Mr. Bergstrom?,wheres mr bergstrom,3.0,f
2,9551,32,211,Miss Hoover: I don't know. Although I'd sure l...,856000,True,464,3.0,Miss Hoover,Springfield Elementary School,I don't know. Although I'd sure like to talk t...,i dont know although id sure like to talk to h...,22.0,f
3,9552,32,212,Lisa Simpson: That life is worth living.,864000,True,9,3.0,Lisa Simpson,Springfield Elementary School,That life is worth living.,that life is worth living,5.0,f
4,9553,32,213,Edna Krabappel-Flanders: The polls will be ope...,864000,True,40,3.0,Edna Krabappel-Flanders,Springfield Elementary School,The polls will be open from now until the end ...,the polls will be open from now until the end ...,33.0,f


In [4]:
# create new df reducing to column for further preprocessing and my y-column

df_reduced = df[['normalized_text','gender']]
                
df_reduced.head()

,normalized_text,gender
0,no actually it was a little of both sometimes ...,f
1,wheres mr bergstrom,f
2,i dont know although id sure like to talk to h...,f
3,that life is worth living,f
4,the polls will be open from now until the end ...,f


In [5]:
df_reduced.dropna(inplace=True)

/Users/Driz/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [ ]:
# need to remove stop words
import nltk
from nltk.corpus import stopwords

stop_words = nltk.corpus.stopwords.words('english')
# stop_words = stopwords.words('english')
filtered_sentence = [w for w in df_reduced['normalized_text'].values if not w in stop_words]
#print filtered_sentence 

In [ ]:
stop_words

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer


tfidf = TfidfVectorizer(stop_words='english',
                       ngram_range=(1,3))
tfidf_matrix = tfidf.fit_transform(df_reduced['normalized_text'].values)


<92692x576162 sparse matrix of type '<type 'numpy.float64'>'
	with 1117934 stored elements in Compressed Sparse Row format>

In [ ]:
from sklearn.decomposition import TruncatedSVD
tsvd = TruncatedSVD(n_components=500000)

new_cols = tsvd.fit_transform(tfidf_matrix)
new_cols.shape

In [27]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

plt.plot(np.cumsum(tsvd.explained_variance_ratio_))

array([ 0.00398391,  0.00310671])

In [7]:
from sklearn.preprocessing import LabelEncoder

y = LabelEncoder().fit_transform(df_reduced['gender'].values)

In [8]:
import numpy as np
np.mean(y)

0.74062486514478054

In [9]:
from sklearn.cross_validation import train_test_split

x_train, x_test, y_train, y_test = train_test_split(tfidf_matrix, y)

In [ ]:
# do label encoder before fed to train test

In [ ]:
# these shapes pertain the filtered_sentence. there is discrepancy in the shape causing it to not work

df_reduced.shape

In [ ]:
tfidf_matrix.shape

In [ ]:
len(filtered_sentence)

In [14]:
from sklearn.ensemble import RandomForestClassifier
# max_features=0.75, max_leaf_nodes=10, max_depth=20, score=0.750000
# these params came from my stalled GridSearchCV below

rf = RandomForestClassifier(max_features=0.75, max_leaf_nodes=10,
                            max_depth=20, n_jobs=-1)
rf.fit(x_train, y_train)
predictions = rf.predict(x_test)
rf.score(x_test, y_test)

0.74617011176800585

In [15]:
# rf.score(predictions, y_test)
predictions.shape

(23173,)

In [16]:
y_test.shape

(23173,)

In [17]:
predictions

array([1, 1, 1, ..., 1, 1, 1])

In [ ]:
rf

In [18]:
from sklearn.metrics import confusion_matrix
#import matplotlib.pyplot as plt --to make the fancy style confusion matrix

confusion = pd.DataFrame(confusion_matrix(y_test, predictions), columns=['predicted_f', 'predicted_m'], index=['is_f', 'is_m'])
confusion

,predicted_f,predicted_m
is_f,520,5530
is_m,352,16771


Looks like my model overly predicts male characters...

Need to run GridSearch over Random Forest model to determine hyperparams

In [10]:
# import all the things!

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, ExtraTreesRegressor, ExtraTreesClassifier, BaggingRegressor,BaggingClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import AdaBoostClassifier, AdaBoostRegressor, GradientBoostingRegressor ,GradientBoostingClassifier
from sklearn.externals.six import StringIO
from sklearn.tree import export_graphviz
import pydot
from IPython.display import Image
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV

In [19]:
# try grid search to seek out some better params
# get rid of depths 150 & 200

params = {'max_depth' : [5, 20, 50],
          'max_features' : [0.25, 0.5, 0.75, 1.0],
          'max_leaf_nodes' : [2,6,10]}

In [21]:
# try GridSearchCV on Random Forest

grid_search_random_forest = GridSearchCV(estimator=RandomForestClassifier(),
                                        param_grid=params, verbose=10)

rf_function = RandomForestClassifier()

grid_search_random_forest.fit(x_train, y_train)

Fitting 3 folds for each of 36 candidates, totalling 108 fits
[CV] max_features=0.25, max_leaf_nodes=2, max_depth=5 ................
[CV]  max_features=0.25, max_leaf_nodes=2, max_depth=5, score=0.745404 - 1.7min
[CV] max_features=0.25, max_leaf_nodes=2, max_depth=5 ................


[Parallel(n_jobs=1)]: Done   1 tasks       | elapsed:  1.7min


[CV]  max_features=0.25, max_leaf_nodes=2, max_depth=5, score=0.741207 - 1.7min
[CV] max_features=0.25, max_leaf_nodes=2, max_depth=5 ................
[CV]  max_features=0.25, max_leaf_nodes=2, max_depth=5, score=0.741196 - 1.7min
[CV] max_features=0.25, max_leaf_nodes=6, max_depth=5 ................
[CV]  max_features=0.25, max_leaf_nodes=6, max_depth=5, score=0.745491 - 4.9min
[CV] max_features=0.25, max_leaf_nodes=6, max_depth=5 ................


[Parallel(n_jobs=1)]: Done   4 tasks       | elapsed:  9.9min


[CV]  max_features=0.25, max_leaf_nodes=6, max_depth=5, score=0.745911 - 5.0min
[CV] max_features=0.25, max_leaf_nodes=6, max_depth=5 ................
[CV]  max_features=0.25, max_leaf_nodes=6, max_depth=5, score=0.746030 - 5.0min
[CV] max_features=0.25, max_leaf_nodes=10, max_depth=5 ...............
[CV]  max_features=0.25, max_leaf_nodes=10, max_depth=5, score=0.745534 - 5.0min
[CV] max_features=0.25, max_leaf_nodes=10, max_depth=5 ...............


...

[Parallel(n_jobs=1)]: Done  17 tasks       | elapsed: 94.9min


[CV]  max_features=0.5, max_leaf_nodes=10, max_depth=5, score=0.749439 -10.2min
[CV] max_features=0.75, max_leaf_nodes=2, max_depth=5 ................
[CV]  max_features=0.75, max_leaf_nodes=2, max_depth=5, score=0.745404 - 5.1min
[CV] max_features=0.75, max_leaf_nodes=2, max_depth=5 ................
[CV]  max_features=0.75, max_leaf_nodes=2, max_depth=5, score=0.745566 - 5.1min
[CV] max_features=0.75, max_leaf_nodes=2, max_depth=5 ................
[CV]  max_features=0.75, max_leaf_nodes=2, max_depth=5, score=0.746030 - 5.1min
[CV] max_features=0.75, max_leaf_nodes=6, max_depth=5 ................
[CV]  max_features=0.75, max_leaf_nodes=6, max_depth=5, score=0.744887 -14.9min
[CV] max_features=0.75, max_leaf_nodes=6, max_depth=5 ................
[CV]  max_features=0.75, max_leaf_nodes=6, max_depth=5, score=0.746386 -14.8min
[CV] max_features=0.75, max_leaf_nodes=6, max_depth=5 ................
[CV]  max_features=0.75, max_leaf_nodes=6, max_depth=5, score=0.747583 -15.1min
[CV] max_featu

[Parallel(n_jobs=1)]: Done  24 tasks       | elapsed: 165.2min


[CV]  max_features=0.75, max_leaf_nodes=10, max_depth=5, score=0.746742 -15.1min
[CV] max_features=0.75, max_leaf_nodes=10, max_depth=5 ...............
[CV]  max_features=0.75, max_leaf_nodes=10, max_depth=5, score=0.747206 -15.1min
[CV] max_features=0.75, max_leaf_nodes=10, max_depth=5 ...............


KeyboardInterrupt: 

In [ ]:
grid_search_model = GridSearchCV(estimator=DecisionTreeRegressor(),
                                 param_grid = params)

decision_function = DecisionTreeRegressor()

grid_search_model.fit(x_train, y_train)

In [ ]:
# which params were optimal?
# looks like the best model went to a depth of 50 (not too many), used 9 features, had 2 leaf nodes(male or female),
# and used best split over random split

grid_search_model.best_estimator_


In [ ]:
# ouch, thats atrocious! what a piece of junk model
# might abandon decision tree and move back to random forest

grid_search_model.best_score_

In [ ]:
# shows how model got to its crappy predictions
# what would I sub for feature_names ????

dot_data = StringIO()  
export_graphviz(grid_search_model.best_estimator_, out_file=dot_data,  
                feature_names=feature_names,  
                filled=True, rounded=True,  
                special_characters=True)  
graph = pydot.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png()) 

Try GridSearch for Random Forest

In [13]:
grid_search_model = GridSearchCV(estimator=RandomForestClassifier(), 
                                param_grid=params, cv=3, n_jobs=-1,
                                verbose=10)

random_forest_function = RandomForestClassifier()

grid_search_model.fit(x_train, y_train)

Fitting 3 folds for each of 36 candidates, totalling 108 fits
[CV] max_features=0.25, max_leaf_nodes=2, max_depth=5 ................
[CV] max_features=0.25, max_leaf_nodes=2, max_depth=5 ................
[CV] max_features=0.25, max_leaf_nodes=2, max_depth=5 ................
[CV] max_features=0.25, max_leaf_nodes=6, max_depth=5 ................
[CV]  max_features=0.25, max_leaf_nodes=2, max_depth=5, score=0.741262 - 3.0min
[CV] max_features=0.25, max_leaf_nodes=6, max_depth=5 ................
[CV]  max_features=0.25, max_leaf_nodes=2, max_depth=5, score=0.745566 - 3.0min
[CV] max_features=0.25, max_leaf_nodes=6, max_depth=5 ................
[CV]  max_features=0.25, max_leaf_nodes=2, max_depth=5, score=0.741239 - 3.0min
[CV] max_features=0.25, max_leaf_nodes=10, max_depth=5 ...............
[CV]  max_features=0.25, max_leaf_nodes=6, max_depth=5, score=0.745577 - 8.7min
[CV] max_features=0.25, max_leaf_nodes=10, max_depth=5 ...............
[CV]  max_features=0.25, max_leaf_nodes=6, max_dep

[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed: 13.0min


[CV]  max_features=0.25, max_leaf_nodes=6, max_depth=5, score=0.746030 -10.1min
[CV] max_features=0.5, max_leaf_nodes=2, max_depth=5 .................
[CV]  max_features=0.25, max_leaf_nodes=10, max_depth=5, score=0.746828 -10.2min
[CV] max_features=0.5, max_leaf_nodes=2, max_depth=5 .................
[CV]  max_features=0.25, max_leaf_nodes=10, max_depth=5, score=0.747422 -10.5min
[CV] max_features=0.5, max_leaf_nodes=2, max_depth=5 .................
[CV]  max_features=0.5, max_leaf_nodes=2, max_depth=5, score=0.745404 - 6.3min
[CV] max_features=0.5, max_leaf_nodes=6, max_depth=5 .................
[CV]  max_features=0.5, max_leaf_nodes=2, max_depth=5, score=0.741251 - 6.3min
[CV] max_features=0.5, max_leaf_nodes=6, max_depth=5 .................


[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed: 19.4min


[CV]  max_features=0.25, max_leaf_nodes=10, max_depth=5, score=0.747411 - 9.1min
[CV] max_features=0.5, max_leaf_nodes=6, max_depth=5 .................
[CV]  max_features=0.5, max_leaf_nodes=2, max_depth=5, score=0.745987 - 6.1min
[CV] max_features=0.5, max_leaf_nodes=10, max_depth=5 ................
[CV]  max_features=0.5, max_leaf_nodes=6, max_depth=5, score=0.746958 -16.0min
[CV] max_features=0.5, max_leaf_nodes=10, max_depth=5 ................
[CV]  max_features=0.5, max_leaf_nodes=6, max_depth=5, score=0.746300 -16.0min
[CV] max_features=0.5, max_leaf_nodes=10, max_depth=5 ................
[CV]  max_features=0.5, max_leaf_nodes=6, max_depth=5, score=0.748101 -15.8min
[CV] max_features=0.75, max_leaf_nodes=2, max_depth=5 ................
[CV]  max_features=0.5, max_leaf_nodes=10, max_depth=5, score=0.749159 -15.1min
[CV] max_features=0.75, max_leaf_nodes=2, max_depth=5 ................
[CV]  max_features=0.75, max_leaf_nodes=2, max_depth=5, score=0.745404 - 7.8min
[CV] max_features

...

[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed: 164.9min


[CV]  max_features=0.75, max_leaf_nodes=2, max_depth=20, score=0.745652 - 6.1min
[CV] max_features=0.75, max_leaf_nodes=6, max_depth=20 ...............
[CV]  max_features=0.75, max_leaf_nodes=2, max_depth=20, score=0.745987 - 6.2min
[CV] max_features=0.75, max_leaf_nodes=6, max_depth=20 ...............
[CV]  max_features=0.5, max_leaf_nodes=10, max_depth=20, score=0.747249 -19.1min
[CV] max_features=0.75, max_leaf_nodes=6, max_depth=20 ...............
[CV]  max_features=0.5, max_leaf_nodes=10, max_depth=20, score=0.748619 -19.3min
[CV] max_features=0.75, max_leaf_nodes=10, max_depth=20 ..............
[CV]  max_features=0.75, max_leaf_nodes=6, max_depth=20, score=0.747001 -21.3min
[CV] max_features=0.75, max_leaf_nodes=10, max_depth=20 ..............
[CV]  max_features=0.75, max_leaf_nodes=6, max_depth=20, score=0.744617 -21.3min
[CV] max_features=0.75, max_leaf_nodes=10, max_depth=20 ..............
[CV]  max_features=0.75, max_leaf_nodes=6, max_depth=20, score=0.748015 -22.2min
[CV] ma

[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed: 221.1min


[CV]  max_features=0.75, max_leaf_nodes=10, max_depth=20, score=0.746256 -33.6min
[CV] max_features=1.0, max_leaf_nodes=6, max_depth=20 ................
[CV]  max_features=0.75, max_leaf_nodes=10, max_depth=20, score=0.750000 -33.7min
[CV] max_features=1.0, max_leaf_nodes=10, max_depth=20 ...............
[CV]  max_features=1.0, max_leaf_nodes=6, max_depth=20, score=0.744498 -29.0min
[CV] max_features=1.0, max_leaf_nodes=10, max_depth=20 ...............
[CV]  max_features=1.0, max_leaf_nodes=6, max_depth=20, score=0.744487 -28.7min
[CV] max_features=1.0, max_leaf_nodes=10, max_depth=20 ...............
[CV]  max_features=1.0, max_leaf_nodes=6, max_depth=20, score=0.747583 -28.9min
[CV] max_features=0.25, max_leaf_nodes=2, max_depth=50 ...............
[CV]  max_features=0.25, max_leaf_nodes=2, max_depth=50, score=0.741175 - 2.4min
[CV] max_features=0.25, max_leaf_nodes=2, max_depth=50 ...............
[CV]  max_features=0.25, max_leaf_nodes=2, max_depth=50, score=0.741251 - 2.4min
[CV] max

KeyboardInterrupt: 

In [ ]:
grid_search_model.best_estimator_

In [ ]:
grid_search_model.fit

In [ ]:
grid_search_model.best_score_

In [ ]:
rf = RandomForestClassifier(n_jobs=-1)
et = ExtraTreesClassifier(n_jobs=-1)

In [ ]:
# want to build a global dict to store all my models for future use
global_models = {
            'rf': RandomForestClassifier(n_jobs=-1),
            'et': ExtraTreesClassifier(n_jobs=-1)
}    

In [ ]:
# comment out predictions line with test set

results = {}

def evaluate_model(x_train, y_train,
                  model):
    pick_a_model = global_models[model]
    pick_a_model.fit(x_train, y_train)
    print 'train model score:', pick_a_model.score(x_train, y_train)
#     predictions = pick_a_model.predict(x_test)
#     print '\nconfusion matrix: \n\n', confusion_matrix(y_test, predictions)
#     print '\nclassification report: \n\n', classification_report(y_test, predictions)
#     results.update({model: pick_a_model.score(X_test, y_test)})

In [ ]:
# looks like random forest is overfit, while extra trees could be a possible model!

for key in global_models:
    evaluate_model(x_train, y_train, key)

In [ ]:
# going to run et model on test data now to see how well it predicts 

results_final = {}

def evaluate_model_final(X_train, X_test, 
                         y_train, y_test,
                         model):
    pick_a_model = global_models[model]
    pick_a_model.fit(x_train, y_train)
    print 'test model score:', pick_a_model.score(x_test, y_test)
    predictions = pick_a_model.predict(x_test)
    print '\nconfusion matrix: \n\n', confusion_matrix(y_test, predictions)
    print '\nclassification report: \n\n', classification_report(y_test, predictions)
    results.update({model: pick_a_model.score(x_test, y_test)})

In [ ]:
# actually a test model set
for key in global_models:
    evaluate_model(x_test, y_test, key)

In [ ]:
# get baseline for model comparison

take # of lines spoken by female character divide # of total rows